In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.metrics import jaccard_score, accuracy_score

In [2]:
# Function to apply all 4 segmentation methods
def apply_segmentation_methods(image):
    methods = {}

    # 1. Canny Edge Detection
    canny = cv2.Canny(image, 100, 200)
    methods["Canny"] = (canny > 0).astype(np.uint8)

    # 2. Canny + Otsu Threshold
    otsu_thresh = threshold_otsu(image)
    binary = (image > otsu_thresh).astype(np.uint8)
    canny_otsu = np.logical_and(canny > 0, binary > 0).astype(np.uint8)
    methods["Canny+Otsu"] = canny_otsu

    # 3. Morphological Post-processing
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(canny_otsu.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
    methods["Morphological"] = morph

    # 4. Refined Contour-based Segmentation
    refined_mask = np.zeros_like(image, dtype=np.uint8)
    clean = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    clean = cv2.morphologyEx(clean, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        hull = cv2.convexHull(largest)
        cv2.drawContours(refined_mask, [hull], -1, 255, -1)
    methods["Refined"] = (refined_mask > 127).astype(np.uint8)

    return methods

# Set your input directory path
input_dir = "your/image/folder/path"
output_dir = "output_folder"
os.makedirs(output_dir, exist_ok=True)

# Process each image
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_dir, filename)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        results = apply_segmentation_methods(image)
        results["Original"] = image

        # Save combined output
        fig, axes = plt.subplots(1, 5, figsize=(20, 4))
        for i, (title, mask) in enumerate(results.items()):
            axes[i].imshow(mask * 255 if title != "Original" else mask, cmap='gray')
            axes[i].set_title(title)
            axes[i].axis('off')
        plt.tight_layout()
        save_path = os.path.join(output_dir, f"segmentation_{filename}")
        plt.savefig(save_path)
        plt.close()

In [3]:
# Function to compute IoU and Dice Score
def calculate_metrics(pred_mask, true_mask):
    pred_mask = pred_mask.flatten() // 255
    true_mask = true_mask.flatten() // 255

    intersection = np.sum(pred_mask * true_mask)
    union = np.sum(pred_mask) + np.sum(true_mask) - intersection

    iou = intersection / union if union != 0 else 0
    dice = (2 * intersection) / (np.sum(pred_mask) + np.sum(true_mask)) if (np.sum(pred_mask) + np.sum(true_mask)) != 0 else 0
    accuracy = accuracy_score(true_mask, pred_mask)

    return iou, dice, accuracy


In [5]:
def process_directory(base_path):
    masked_images_path = os.path.join(base_path, "1", "face_crop")
    ground_truth_path = os.path.join(base_path, "1", "face_crop_segmentation")

    image_files = [f for f in os.listdir(masked_images_path) if f.endswith(('.jpg', '.png'))]

    results = []
    for file_name in image_files:
        img_path = os.path.join(masked_images_path, file_name)
        mask_path = os.path.join(ground_truth_path, file_name)

        if not os.path.exists(mask_path):
            continue  # Skip if no corresponding ground truth

        # Read images and masks
        image = cv2.imread(img_path)
        true_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Segment the mask
        pred_mask = segment_mask(image)

        # Compute metrics
        iou, dice, acc = calculate_metrics(pred_mask, true_mask)

        results.append([file_name, iou, dice, acc])

    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["Image", "IoU", "Dice Score", "Accuracy"])

    return df_results

In [ ]:
base_directory = "path_to_mfsd_directory"
results_df = process_directory(base_directory)
print(results_df)